In [1]:
!nvidia-smi


Sun Dec 29 12:10:01 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    33W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# import pdb

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

data_root = '/gdrive/My Drive/data_tf'
# Above is for notebook only
# Below is to be moved to py files
! ls '/gdrive/My Drive/python-code/文本摘要-01'
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
checkpoints	   result      work-tf-unigru-attn-beam.ipynb
data		   utils       work-tf-unigru-attn.ipynb
pgn_batcher.ipynb  work.ipynb  work-tf-unigru-attn-pgn.ipynb
TensorFlow 2.x selected.


In [26]:
%%shell

cd '/gdrive/My Drive/python-code/文本摘要-01'
pwd

git config --global user.email "luopub@live.com"

echo "# textsummary" >> README.md
git init
git add README.md
git commit -m "first commit"
git remote add origin https://github.com/luopub/textsummary.git
git push -u origin master

/gdrive/My Drive/python-code/文本摘要-01
Reinitialized existing Git repository in /gdrive/My Drive/python-code/文本摘要-01/.git/
[master 7210809] first commit
 1 file changed, 1 insertion(+)
fatal: could not read Username for 'https://github.com': No such device or address


CalledProcessError: ignored

In [3]:
# load the autoreload extension
%load_ext autoreload
# Set extension to reload modules every time before executing code
%autoreload 2

# import modules
import os
import sys
import pathlib
import pandas as pd
import numpy as np
import pdb
import time
import tensorflow as tf

project_root = pathlib.Path(r'/gdrive/My Drive/python-code/文本摘要-01')
sys.path.append(str(project_root))

from utils.config import *
from utils.multi_proc_utils import *
from utils.data_loader import *
from utils.file_utils import *
from utils.data_preprocessor import *

tf.config.experimental_run_functions_eagerly(True)

Building prefix dict from the default dictionary ...
2019-12-29 12:10:08,884 : DEBUG : Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
2019-12-29 12:10:08,892 : DEBUG : Loading model from cache /tmp/jieba.cache
Loading model cost 0.749 seconds.
2019-12-29 12:10:09,641 : DEBUG : Loading model cost 0.749 seconds.
Prefix dict has been built succesfully.
2019-12-29 12:10:09,645 : DEBUG : Prefix dict has been built succesfully.


In [0]:
# define batcher keys
bkey_abstract = "abstract"
bkey_article = "article"
bkey_article_oovs = "article_oovs"
bkey_dec_input = "dec_input"
bkey_dec_len = "dec_len"
bkey_dec_target = "dec_target"
bkey_decoder_pad_mask = "decoder_pad_mask"
bkey_enc_input = "enc_input"
bkey_enc_len = "enc_len"
bkey_encoder_pad_mask = "encoder_pad_mask"
bkey_extended_enc_input = "extended_enc_input"
bkey_max_oov_len = "max_oov_len"
bkey_target = "target"

tf.config.experimental_run_functions_eagerly(True)
def article_to_ids(article_words, vocab):
  ids = []
  oovs = []
  unk_id = vocab.token_id_unk
  for w in article_words:
    i = vocab.word_to_id(w)
    if i == unk_id:  # If w is OOV
      if w not in oovs:  # Add to list of OOVs
        oovs.append(w)
      oov_num = oovs.index(w)  # This is 0 for the first article OOV, 1 for the second article OOV...
      ids.append(vocab.vocab_size + oov_num)  # This is e.g. 50000 for the first article OOV, 50001 for the second...
    else:
      ids.append(i)
  return ids, oovs


def abstract_to_ids(abstract_words, vocab, article_oovs):
  ids = []
  unk_id = vocab.token_id_unk
  for w in abstract_words:
    i = vocab.word_to_id(w)
    if i == unk_id:  # If w is an OOV word
      if w in article_oovs:  # If w is an in-article OOV
        vocab_idx = vocab.vocab_size + article_oovs.index(w)  # Map to its temporary article OOV number
        ids.append(vocab_idx)
      else:  # If w is an out-of-article OOV
        ids.append(unk_id)  # Map to the UNK token id
    else:
      ids.append(i)
  return ids


def output_to_words(id_list, vocab, article_oovs):
  words = []
  for i in id_list:
    if i < vocab.vocab_size:
      w = vocab.id_to_word(i)  # might be [UNK]
    # except ValueError as e:  # w is OOV
    else:
      # assert article_oovs is not None, "Error: model produced a word ID that isn't in the vocabulary. \
      # This should not happen in baseline (no pointer-generator) mode"
      article_oov_idx = i - vocab.vocab_size
      if article_oov_idx < len(article_oovs):
        w = article_oovs[article_oov_idx]
      # except ValueError as e:  # i doesn't correspond to an article oov
      #   raise ValueError(
      else:
        print('Error: model produced word ID %i which corresponds \
           to article OOV %i but this example only has %i article OOVs' % (
            i, article_oov_idx, len(article_oovs)))
        w = vocab.WORD_UNK
    words.append(w)
  return words

def add_seq_start_stop(sequence, max_len, start_id, stop_id):
  inp_len = len(sequence)
  max_len -= 2

  if inp_len > max_len:  # truncate
    inp = sequence[:max_len]
  else:
    inp = sequence[:]

  return inp, [start_id] + inp + [stop_id]

def get_dec_inp_targ_seqs(sequence, max_len, start_id, stop_id):
  """
  Given the reference summary as a sequence of tokens, return the input sequence for the decoder, and the target sequence which we will use to calculate loss. The sequence will be truncated if it is longer than max_len. The input sequence must start with the start_id and the target sequence must end with the stop_id (but not if it's been truncated).
  Args:
    sequence: List of ids (integers)
    max_len: integer
    start_id: integer
    stop_id: integer
  Returns:
    inp: sequence length <=max_len starting with start_id
    target: sequence same length as input, ending with stop_id only if there was no truncation
  """

  return add_seq_start_stop(sequence, max_len, start_id, stop_id)


def get_enc_inp_targ_seqs(sequence, max_len, start_id, stop_id):
  """
  Given the reference summary as a sequence of tokens, return the input sequence for the decoder, and the target sequence which we will use to calculate loss. The sequence will be truncated if it is longer than max_len. The input sequence must start with the start_id and the target sequence must end with the stop_id (but not if it's been truncated).
  Args:
    sequence: List of ids (integers)
    max_len: integer
    start_id: integer
    stop_id: integer
  Returns:
    inp: sequence length <=max_len starting with start_id
    target: sequence same length as input, ending with stop_id only if there was no truncation
  """
  return add_seq_start_stop(sequence, max_len, start_id, stop_id)

# pdb.set_trace()
def doc_tokenize(vocab, doc, max_len, oovs):
  doc_words = doc.split()[:max_len-2]
  tokens = [vocab.word_to_id(w) for w in doc_words]

  if not oovs:
    extend_tokens, oovs = article_to_ids(doc_words, vocab)
  else:
    extend_tokens = abstract_to_ids(doc_words, vocab, oovs)

  # add start and stop flag
  _, tokens = add_seq_start_stop(tokens, max_len, vocab.token_id_start, vocab.token_id_stop)
  _, extend_tokens = add_seq_start_stop(extend_tokens, max_len, vocab.token_id_start, vocab.token_id_stop)

  # 添加mark标记
  pad_mask = [1 for _ in range(len(tokens))]

  return tokens, extend_tokens, pad_mask, oovs

def example_generator(vocab, max_enc_len, max_dec_len, mode, batch_size):
  if mode == "train":
    dataset = tf.data.experimental.make_csv_dataset(
        train_seg_path, batch_size=2, select_columns=['Question', 'Dialogue', 'Report', 'QID'])
    # Here should be noted:
    # If map() returns two value, then 'for x in dataset' get a single record with two value each time
    # If map() returns one value, then 'for x in dataset' get batch_size records with each time
    dataset = dataset.map(lambda x: (tf.strings.join([x['Question'],x['Dialogue']], separator=' '), x['Report']))
    dataset = dataset.shuffle(10, reshuffle_each_iteration=True)

    for raw_record in dataset:
      article = raw_record[0].numpy()[0].decode("utf-8")

      enc_input, extended_enc_input, sample_encoder_pad_mask, article_oovs = doc_tokenize(vocab, article, max_enc_len, None)

      abstract = raw_record[1].numpy()[0].decode("utf-8")

      dec_input, target, sample_decoder_pad_mask, article_oovs = doc_tokenize(vocab, abstract, max_dec_len, article_oovs)

      output = {
        bkey_enc_len: len(enc_input),
        bkey_enc_input: enc_input,
        bkey_extended_enc_input: extended_enc_input,
        bkey_article_oovs: article_oovs,
        bkey_dec_input: dec_input,
        bkey_target: target,
        bkey_dec_len: len(dec_input),
        bkey_article: article,
        bkey_abstract: abstract,
        bkey_decoder_pad_mask: sample_decoder_pad_mask,
        bkey_encoder_pad_mask: sample_encoder_pad_mask
      }
      yield output
  else:
    dataset = tf.data.experimental.make_csv_dataset(
        test_seg_path, batch_size=batch_size, select_columns=['Question', 'Dialogue', 'QID'])
    dataset = dataset.map(lambda x: (tf.strings.join([x['Question'],x['Dialogue']], separator=' ')))
    dataset = dataset.shuffle(10, reshuffle_each_iteration=True)

    for raw_record in dataset:
      for i in range(len(raw_record)):
        article = raw_record[i].numpy().decode("utf-8")

        enc_input, extended_enc_input, sample_encoder_pad_mask, article_oovs = doc_tokenize(vocab, article, max_enc_len, None)

        output = {
          bkey_enc_len: len(enc_input),
          bkey_enc_input: enc_input,
          bkey_extended_enc_input: extended_enc_input,
          bkey_article_oovs: article_oovs,
          bkey_article: article,
          bkey_encoder_pad_mask: sample_encoder_pad_mask
        }
        yield output


def batch_generator(generator, vocab, max_enc_len, max_dec_len, batch_size, mode):
  # pdb.set_trace()
  output_types_enc={
    bkey_enc_len: tf.int32,
    bkey_enc_input: tf.int32,
    bkey_extended_enc_input: tf.int32,
    bkey_article_oovs: tf.string,
    bkey_article: tf.string,
    bkey_encoder_pad_mask: tf.int32,
  }
  output_types_dec={
    bkey_dec_input: tf.int32,
    bkey_target: tf.int32,
    bkey_dec_len: tf.int32,
    bkey_abstract: tf.string,
    bkey_decoder_pad_mask: tf.int32,
  }
  output_shapes_enc={
    bkey_enc_len: [],
    bkey_enc_input: [None],
    bkey_extended_enc_input: [None],
    bkey_article_oovs: [None],
    bkey_article: [],
    bkey_encoder_pad_mask: [None]
  }
  output_shapes_dec={
    bkey_dec_input: [None],
    bkey_target: [None],
    bkey_dec_len: [],
    bkey_abstract: [],
    bkey_decoder_pad_mask: [None],
  }
  padded_shapes_enc={
    bkey_enc_len: [],
    bkey_enc_input: [max_enc_len],
    bkey_extended_enc_input: [max_enc_len],
    bkey_article_oovs: [None],
    bkey_article: [],
    bkey_encoder_pad_mask: [max_enc_len]
  }
  padded_shapes_dec={
    bkey_dec_input: [max_dec_len],
    bkey_target: [max_dec_len],
    bkey_dec_len: [],
    bkey_abstract: [],
    bkey_decoder_pad_mask: [max_dec_len],
  }
  padding_values_enc={
    bkey_enc_len: -1,
    bkey_enc_input: vocab.word_to_id(vocab.WORD_PAD),
    bkey_extended_enc_input: vocab.word_to_id(vocab.WORD_PAD),
    bkey_article_oovs: b'',
    bkey_article: b"",
    bkey_encoder_pad_mask: 0
  }
  padding_values_dec={
    bkey_dec_input: vocab.word_to_id(vocab.WORD_PAD),
    bkey_target: vocab.word_to_id(vocab.WORD_PAD),
    bkey_dec_len: -1,
    bkey_abstract: b"",
    bkey_decoder_pad_mask: 0,
  }

  if mode == 'train':
    output_types = {**output_types_enc, **output_types_dec}
    output_shapes = {**output_shapes_enc, **output_shapes_dec}
    padded_shapes = {**padded_shapes_enc, **padded_shapes_dec}
    padding_values = {**padding_values_enc, **padding_values_dec}
  else:
    output_types = output_types_enc
    output_shapes = output_shapes_enc
    padded_shapes = padded_shapes_enc
    padding_values = padding_values_enc

  dataset = tf.data.Dataset.from_generator(
          lambda: generator(vocab, max_enc_len, max_dec_len, mode, batch_size),
          output_types=output_types,
          output_shapes=output_shapes)

  dataset = dataset.padded_batch(batch_size,
          padded_shapes=(padded_shapes),
          padding_values=padding_values,
          drop_remainder=True)

  def update(entry):
    record_enc = {
          bkey_enc_input: entry[bkey_enc_input],
          bkey_extended_enc_input: entry[bkey_extended_enc_input],
          bkey_article_oovs: entry[bkey_article_oovs],
          bkey_enc_len: entry[bkey_enc_len],
          bkey_article: entry[bkey_article],
          bkey_max_oov_len: tf.shape(entry[bkey_article_oovs])[1],
          bkey_encoder_pad_mask: entry[bkey_encoder_pad_mask]
    }
    if mode == 'train':
      record_dec = {
            bkey_dec_input: entry[bkey_dec_input],
            bkey_dec_target: entry[bkey_target],
            bkey_dec_len: entry[bkey_dec_len],
            bkey_abstract: entry[bkey_abstract],
            bkey_decoder_pad_mask: entry[bkey_decoder_pad_mask]
      }
      return (record_enc, record_dec)
    else:
      return (record_enc, )

  dataset = dataset.map(update)
  return dataset

def batcher(working_ds):
  tf.config.experimental_run_functions_eagerly(True)
  dataset = batch_generator(example_generator,
                working_ds,
                working_ds.X_max_len ,
                working_ds.Y_max_len,
                BATCH_SIZE,
                mode)
  return dataset



In [5]:
# %%timeit -n1 -r1

mode = 'train'
use_pgn = True
BATCH_SIZE = 16

working_ds = AutoCarDataSet()
# %prun 
working_ds.prepare_data(force_build=False)
# %prun 
working_ds.get_wv_model(force_build=False)


File read:  /gdrive/My Drive/python-code/文本摘要-01/data/train_seg_data.csv /gdrive/My Drive/python-code/文本摘要-01/data/test_seg_data.csv
sentences_proced train data size 82823,test data size 20000


2019-12-29 12:10:12,940 : INFO : loading Word2Vec object from /gdrive/My Drive/python-code/文本摘要-01/data/wv/word2vec.model


File read:  /gdrive/My Drive/python-code/文本摘要-01/data/merged_train_test_seg_data.csv


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-12-29 12:10:13,728 : INFO : loading wv recursively from /gdrive/My Drive/python-code/文本摘要-01/data/wv/word2vec.model.wv.* with mmap=None
2019-12-29 12:10:13,730 : INFO : setting ignored attribute vectors_norm to None
2019-12-29 12:10:13,732 : INFO : loading vocabulary recursively from /gdrive/My Drive/python-code/文本摘要-01/data/wv/word2vec.model.vocabulary.* with mmap=None
2019-12-29 12:10:13,734 : INFO : loading trainables recursively from /gdrive/My Drive/python-code/文本摘要-01/data/wv/word2vec.model.trainables.* with mmap=None
2019-12-29 12:10:13,736 : INFO : setting ignored attribute cum_table to None
2019-12-29 12:10:13,738 : INFO : l

In [0]:
!ls -l '/gdrive/My Drive/python-code/文本摘要-01/utils/data_preprocessor.py'

In [0]:
tf.config.experimental_run_functions_eagerly(True)

# pdb.set_trace()
b = batcher(working_ds)

# pdb.set_trace()
print(next(iter(b)))


In [0]:
working_ds.__dict__.keys()

In [7]:
print(tf.__version__)
logical_devices = tf.config.list_logical_devices('GPU')
print(logical_devices)
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

2.1.0-rc1
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [38]:
def map_func(x):
  return tf.strings.join([x['Question'],x['Dialogue']], separator=' '), x['Report'], x['QID']
train_dataset = tf.data.experimental.make_csv_dataset(
    train_seg_path, batch_size=2, select_columns=['Question', 'Dialogue', 'Report', 'QID'])
train_dataset = train_dataset.map(lambda x: (tf.strings.join([x['Question'],x['Dialogue']], separator=' '), x['Report']))

for r in train_dataset.take(2):
  print(len(r))
  # print(r[2])
  print(r[0].numpy()[0].decode('utf-8'))
  print(r[1].numpy()[0].decode('utf-8'))
  # print(r['Question'].numpy()[0].decode('utf-8'))
  # print(r['Dialogue'].numpy()[0].decode('utf-8'))
  # print(r['Report'].numpy()[0].decode('utf-8'))
  # print(r['Question'][0].numpy().decode('utf-8'))
  # break

2
私家车 一个月 前 违章 ， 违法 变道 扣 三分 罚款 100 ， 去 处理 违章 ， 交警 告诉 身份证 没磁 ， 需要 换 身份证 。 媳妇 刚考 下来 驾驶证 没 一个月 ， 媳妇 消分 ？ 谢谢 技师说 语音 | 车主说 不是 高清 ， 媳妇 刚考 下来 驾照 没 几天 ， 一个月 前 违章 ， 能消 | 技师说 语音 | 车主说 好 ， 谢谢 | 技师说 客气
， 描述 情况 分析 ， 不 违章 时间 驾驶证 早 ，
2
大众 宝来 后 门拉手 拆 技师说 语音 | 车主说 语音 | 车主说 语音 | 技师说 语音 | 技师说 语音 | 车主说 语音 | 技师说 语音 | 车主说 图片 | 车主说 语音 | 技师说 语音 | 车主说 语音 | 技师说 语音 | 车主说 语音 | 技师说 语音 | 车主说 语音 | 技师说 语音 | 车主说 图片 | 车主说 语音 | 技师说 语音 | 车主说 语音 | 车主说 语音 | 技师说 语音 | 车主说 语音 | 技师说 语音 | 车主说 图片 | 车主说 语音 | 技师说 语音 | 车主说 图片 | 车主说 语音 | 技师说 语音 | 技师说 语音 | 车主说 语音 | 技师说 语音
咨询 问题 ， 说明 拉 把手 拆卸 方法 ， 很 高兴 解答 ！


In [0]:
working_ds.vocab_size

In [23]:
article_words = '起动 后 右 转向灯 工作 金正恩'.split(' ', maxsplit=-1)
print(article_words)
aids,oovs = article_to_ids(article_words, working_ds)
print(aids)
print(oovs)

['起动', '后', '右', '转向灯', '工作', '金正恩']
[780, 41, 514, 1155, 129, 31814]
['金正恩']


In [24]:
aids = abstract_to_ids(article_words, working_ds, oovs)
print(aids)

[780, 41, 514, 1155, 129, 31814]


In [25]:
ows = output_to_words(aids, working_ds, oovs)
print(ows)

['起动', '后', '右', '转向灯', '工作', '金正恩']


In [26]:
inp, target = get_enc_inp_targ_seqs(aids, 8, working_ds.token_id_start, working_ds.token_id_stop)
print(aids)
print(inp)
print(target)


[780, 41, 514, 1155, 129, 31814]
[780, 41, 514, 1155, 129, 31814]
[31810, 780, 41, 514, 1155, 129, 31814, 31811]


In [20]:

batch_size = 16

# def map_func(x):
#   return tf.strings.join([x['Question'],x['Dialogue']], separator=' '), x['Report'], x['QID']

batch_size = 16
# train_dataset = tf.data.experimental.make_csv_dataset(train_seg_path, batch_size=batch_size, select_columns=['Question', 'Dialogue', 'Report'])
# train_dataset = train_ds.map(map_func)
# train_dataset = train_dataset.shuffle(10, reshuffle_each_iteration=True).repeat()
# train_dataset.take(2)

# def map_func(x):
#   return tf.strings.join([x['Question'],x['Dialogue']], separator=' '), x['Report'], x['QID']
train_dataset = tf.data.experimental.make_csv_dataset(
    train_seg_path, batch_size=2, select_columns=['Question', 'Dialogue', 'Report', 'QID'])
train_dataset = train_dataset.map(lambda x: (tf.strings.join([x['Question'],x['Dialogue']], separator=' '), x['Report']))
train_dataset = train_dataset.shuffle(10, reshuffle_each_iteration=True)


for raw_record in train_dataset.take(2):
  print(len(raw_record))
  article = raw_record[0].numpy()[0].decode("utf-8")
  print(article)
  article = raw_record[1].numpy()[0].decode("utf-8")
  print(article)


2
租车 发动机 蹭 坏 保险公司 会 赔 。 维修 费用 划分 。 技师说 ， 车损险 保险公司 理赔 ， 没有 自费 ， 看 租车 合同 上面 ， 有没有 保险
车损险 ， 租车 合同 上面 ， 报 保险 更换 ， 没有 ， 自费 维修
2
请问 小车 加速 无力 ， 加油 慢是 原因 技师说 ， 清洗 下 节气门 | 车主说 反应 好慢 ， 加油 很 久 速度 才 升上去 | 车主说 清洗 节气门 钱 | 技师说 大约 100 元 左右 | 车主说 这款 车 好像 电脑 匹配 ， 小店 做 | 技师说 对要 匹配 修理厂 电脑 匹配 | 车主说 节气门 公里 清洗 一次 | 技师说 大约 2 万公里 清洗 一次 | 车主说 还 问 问题 | 车主说 想 问 一下 本田 飞度 怠速 调低 | 技师说 不能 电脑 控制
检查 下 三元催化 油路 堵不堵


In [0]:
vocab = working_ds
max_enc_len = working_ds.X_max_len
max_dec_len = working_ds.Y_max_len
mode = 'test' #'train' #  
batch_size = 2

In [0]:
dataset = batcher(working_ds)

In [9]:
dataset = batcher(working_ds)

def printoutput(output):
  for k, v in output.items():
    print(k, " : ", v)
  print()
# output_gen = example_generator(vocab, max_enc_len, max_dec_len, mode, batch_size)
output_gen = iter(dataset)
record = next(output_gen)
print(len(record))
for i in range(len(record)):
  print(type(record[i]))
print(record)
# printoutput(record)
# printoutput(next(output_gen))
# printoutput(next(output_gen))
# printoutput(next(output_gen))

1
<class 'dict'>
({'enc_input': <tf.Tensor: shape=(16, 390), dtype=int32, numpy=
array([[31810,    69,   104, ..., 31812, 31812, 31812],
       [31810,   104,   202, ..., 31812, 31812, 31812],
       [31810,     0,   272, ..., 31812, 31812, 31812],
       ...,
       [31810,   113,     0, ..., 31812, 31812, 31812],
       [31810,    20, 15703, ..., 31812, 31812, 31812],
       [31810,  1631,    75, ..., 31812, 31812, 31812]], dtype=int32)>, 'extended_enc_input': <tf.Tensor: shape=(16, 390), dtype=int32, numpy=
array([[31810,    69,   104, ..., 31812, 31812, 31812],
       [31810,   104,   202, ..., 31812, 31812, 31812],
       [31810,     0,   272, ..., 31812, 31812, 31812],
       ...,
       [31810,   113,     0, ..., 31812, 31812, 31812],
       [31810,    20, 15703, ..., 31812, 31812, 31812],
       [31810,  1631,    75, ..., 31812, 31812, 31812]], dtype=int32)>, 'article_oovs': <tf.Tensor: shape=(16, 4), dtype=string, numpy=
array([[b'\xe5\x8a\xa8\xe6\x89\x8b\xe5\x81\x9a', b'', b'

In [0]:
dataset = tf.data.experimental.make_csv_dataset(
    test_seg_path, batch_size=batch_size, select_columns=['Question', 'Dialogue', 'QID'])
dataset = dataset.map(lambda x: (tf.strings.join([x['Question'],x['Dialogue']], separator=' ')))
dataset = dataset.shuffle(10, reshuffle_each_iteration=True)

for raw_record in dataset.take(2):
  print(len(raw_record))
  for i in range(len(raw_record)):
    article = raw_record[i].numpy().decode("utf-8")

    enc_input, enc_input_extend_vocab, sample_encoder_pad_mask, article_oovs = doc_tokenize(vocab, article, max_enc_len, None)

    output = {
      bkey_enc_len: len(enc_input),
      bkey_enc_input: enc_input,
      bkey_enc_input_extend_vocab: enc_input_extend_vocab,
      bkey_article_oovs: article_oovs,
      bkey_dec_input: '',
      bkey_target: '',
      bkey_dec_len: vocab.Y_max_len,
      bkey_article: article,
      bkey_abstract: '',
      bkey_decoder_pad_mask: [],
      bkey_encoder_pad_mask: sample_encoder_pad_mask
    }

    print(enc_input)
    print(enc_input_extend_vocab)
    print(sample_encoder_pad_mask)
    print(article_oovs)
    print()
    # print(output)